<a href="https://colab.research.google.com/github/jmachima/Archaeology_News/blob/main/Archaeology_News_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Jasmine Machima 
jasmine.machima@gmail.com

Text Classification with content from archaeology.org/news (with permission from the editor-in-chief of Archaeology - A publication of the Archaeological Institute of America)

In [ ]:
!pip install -q -U tensorflow-text
!pip install -q tf-models-official

In [2]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text  # A dependency of the preprocessing model
import numpy as np
from sklearn.model_selection import train_test_split
tf.get_logger().setLevel('ERROR')

In [3]:
# TPU will be used for text classification with BERT (Bidirectional Encoder Representations from Transformers).

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
strategy = tf.distribute.TPUStrategy(resolver)

All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]


In [ ]:
!pip install transformers

In [5]:
import pandas as pd
pd.options.display.max_rows = 999
import numpy as np 
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig

In [ ]:
#bert-base-cased
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased', output_attention=True)
# 14 text categories for text classification

In [ ]:
from google.colab import files

# upload ARCHAIC_TrainAndValidate.csv for training and validation

#Upload 2 more files for test data sets
# ARCHAIC_test.csv and ARCHAIC_2labels.csv
files.upload()

In [8]:
data = pd.read_csv('ARCHAIC_TrainAndValidate.csv')

Class = set(data['category'])  # List of unique categories

CL = dict(zip(Class,list(range(14)))) # convert categories into numericals

rvCL = dict(zip(list(range(14)),Class)) # reverse lookup for numerical label using another dictionary

print(CL)

{'Native American': 0, 'South&Southeast Asia': 1, 'Roman': 2, 'Egyptian': 3, 'Bronze Age': 4, 'Neolithic': 5, 'Pre-Human': 6, 'Paleolithic': 7, 'Modern': 8, 'East Asia': 9, 'Iron Age': 10, 'Pre-Columbian': 11, 'Near East': 12, 'Medieval': 13}


In [9]:
# Function for assigning numerical category to news text
#Also check if a news piece has a sequence length longer than 384
def col_label(Table): 
 
  Table['Label'] = 0 # want integers only
  long=[]

  for i in Table.index:
      
      Table.at[i,'Label'] = CL[Table.at[i,'category']]
      tokens=bert_tokenizer.tokenize(Table.at[i,'clean text'])
      if len(tokens) > 384:
        print('i=',i, len(tokens))
        long.append([i,len(tokens)])
      else:
        pass
  return Table, long

# Assign input IDs and attention masks

def Encode(table):

  sentences=table['clean text']
  labels=table['Label']

  input_ids=[]
  attention_masks=[]

  for sent in sentences:
      bert_inp=bert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =384,pad_to_max_length = True,return_attention_mask = True)
      input_ids.append(bert_inp['input_ids'])
      attention_masks.append(bert_inp['attention_mask'])

  input_ids=np.asarray(input_ids)
  attention_masks=np.array(attention_masks)
  labels=np.array(labels)
  return input_ids, attention_masks, labels


# Use argmax to get Label of predicted category (for later use)
# Reverse look-up for category names

def rev_label(pred, X,y_true):

  predicted =[]
  pred_cat = []
  true_cat = []
  for j in range(len(X)):
    index= np.argmax(pred[0][j])
    predicted.append(int(index))
    cat = rvCL[index]
    true = rvCL[y_true[j]]
    pred_cat.append(cat)
    true_cat.append(true)
  return predicted, pred_cat, true_cat

In [10]:
# Add a 'Label' column to the data table
data, over_384 = col_label(data)

i= 819 401


In [11]:
# IDs and masks for news texts

input_ids, attention_masks, labels = Encode(data)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [12]:
# Split input IDs, labels, and masks for training and validation sets

train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.25, random_state=857)
print('Train inp shape {} Val input shape {}\nTrain label shape {} Val label shape {}\nTrain attention mask shape {} Val attention mask shape {}'.format(train_inp.shape,val_inp.shape,train_label.shape,val_label.shape,train_mask.shape,val_mask.shape))

Train inp shape (710, 384) Val input shape (237, 384)
Train label shape (710,) Val label shape (237,)
Train attention mask shape (710, 384) Val attention mask shape (237, 384)


In [13]:
# Training and validation
# With early stopping when criterion is met

with strategy.scope():

  Loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  Metric = tf.keras.metrics.SparseCategoricalAccuracy()
  callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
  Optimizer = tf.keras.optimizers.Adam(learning_rate=0.00000915,epsilon=5e-10)
  bert_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
  bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=14)
  bert_model.compile(loss=Loss, metrics=Metric,optimizer=Optimizer)
  history=bert_model.fit([train_inp,train_mask],train_label,batch_size=64,epochs=100,validation_data=([val_inp,val_mask],val_label),callbacks=[callback])

Downloading:   0%|          | 0.00/527M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/100


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None,) dtype=int64>]
INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None,) dtype=int64>]


12/12 [==============================] - ETA: 0s - loss: 2.6822 - sparse_categorical_accuracy: 0.0676

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None,) dtype=int64>]


12/12 [==============================] - 157s 7s/step - loss: 2.6822 - sparse_categorical_accuracy: 0.0676 - val_loss: 2.6530 - val_sparse_categorical_accuracy: 0.0717
Epoch 2/100
12/12 [==============================] - 3s 268ms/step - loss: 2.6157 - sparse_categorical_accuracy: 0.1056 - val_loss: 2.5943 - val_sparse_categorical_accuracy: 0.1561
Epoch 3/100
12/12 [==============================] - 3s 266ms/step - loss: 2.5445 - sparse_categorical_accuracy: 0.1972 - val_loss: 2.4807 - val_sparse_categorical_accuracy: 0.3038
Epoch 4/100
12/12 [==============================] - 3s 264ms/step - loss: 2.4081 - sparse_categorical_accuracy: 0.3127 - val_loss: 2.3139 - val_sparse_categorical_accuracy: 0.3882
Epoch 5/100
12/12 [==============================] - 3s 267ms/step - loss: 2.2250 - sparse_categorical_accuracy: 0.4521 - val_loss: 2.1285 - val_sparse_categorical_accuracy: 0.4810
Epoch 6/100
12/12 [==============================] - 3s 265ms/step - loss: 2.0466 - sparse_categorical_accur

In [14]:
# Split texts (raw data) into train and validation sets so that validation set can be examined.

X_tr, X_ts, Y_tr, Y_ts=train_test_split(data['clean text'],data['Label'],test_size=0.25, random_state=857)

# Make a table for the validation set alone
val_set = pd.DataFrame(X_ts)
val_set['Label'] = Y_ts
y_true = list(Y_ts)

In [15]:
#Obtain predicted categories from the trained BERT model

pred = bert_model([val_inp,val_mask])

In [16]:
# predicted for predicted label
# pred_cat for predicted category name
# true_cat for actual category
predicted, pred_cat, true_cat = rev_label(pred, X_ts, y_true)

val_set['predicted'] = predicted
val_set['true_cat'] = true_cat
val_set['pred_cat'] = pred_cat

In [17]:
print('Incorrectly predicted news pieces in total:',len(val_set[val_set['Label']!=val_set['predicted']]))
val_set[val_set['Label']!=val_set['predicted']]

Incorrectly predicted news pieces in total: 22


,clean text,Label,predicted,true_cat,pred_cat
232,"Intact Coffins Discovered in Saqqara. CAIRO, E...",3,12,Egyptian,Near East
566,Mithraeum Excavated in Rome’s Port of Ostia. B...,2,3,Roman,Egyptian
24,Possible Insect Rock Art Found in Iran. TEHRAN...,7,5,Paleolithic,Neolithic
465,"Rare 5,000-Year-Old Sword Identified in Venice...",4,10,Bronze Age,Iron Age
827,New Thoughts on the Causes of Angkor’s Decline...,1,13,South&Southeast Asia,Medieval
934,Traces of Nicotine Detected in Ancient Dental ...,0,11,Native American,Pre-Columbian
349,New Thoughts on Turkey’s Oldest Temple Complex...,5,4,Neolithic,Bronze Age
290,Smuggling Attempt Foiled at Egypt’s Port of Al...,12,3,Near East,Egyptian
42,Floodwaters Threaten Sudan’s Ancient Pyramids....,3,4,Egyptian,Bronze Age
706,New Evidence for Hunter-Gatherer Trade in Nort...,0,7,Native American,Paleolithic


In [18]:
# Contains test data set which is somewhat more difficult than any validation set
# Test data set has only 1 label per news story
Easy_test = pd.read_csv('ARCHAIC_test.csv')

In [19]:
# A place to assure there are no duplicates from the training set in the test sets
DUP = []
arch_testing = list(Easy_test['url'])
for j in range(len(data['url'])):
  if data.at[j,'url'] in arch_testing:
    DUP.append(data.at[j,'url'])
    print(j)
  else:
    pass

#If there is anything in DUP, it has to be removed from the test data set.
DUP

[]

In [20]:
# Create a numerical label column for the Easy_test table
Easy_test, Easy_len=col_label(Easy_test)

# encode the texts in Easy_test
test_input_ids, test_attention_masks, test_labels = Encode(Easy_test)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [21]:
# Predicting news category on the test set
Easy_test_pred = bert_model([test_input_ids,test_attention_masks])

# Add predicted label and category columns to the Easy_test table
es_predicted, es_pred_cat, es_true_cat = rev_label(Easy_test_pred, Easy_test['clean text'], test_labels)

Easy_test['predicted'] = es_predicted
Easy_test['true_cat'] = es_true_cat
Easy_test['pred_cat'] = es_pred_cat

In [22]:
print('Wrongly predicted news pieces from test set:',len(Easy_test[Easy_test['Label']!=Easy_test['predicted']]))
Easy_test[Easy_test['Label']!=Easy_test['predicted']][['clean text','category','pred_cat']]

Wrongly predicted news pieces from test set: 129


,clean text,category,pred_cat
0,Nomadic Warriors’ Remains Unearthed in Croatia...,Medieval,Roman
1,"New Study Suggests Measles Virus is 2,500 Year...",Iron Age,Pre-Human
2,Canaanite Fortress Unearthed in Southern Israe...,Bronze Age,Near East
3,Ancient Hue Detected in Renaissance Painting. ...,Medieval,Roman
26,Possible Traces of Antebellum Home Unearthed i...,Modern,Native American
27,Secret Passage Discovered in London’s House of...,Medieval,Modern
34,Soviet Shortwave Spy Radio Unearthed in Wester...,Modern,Roman
39,Researchers Will Search for Spanish Treasure S...,Medieval,Modern
75,16th-Century Manuscript Attributed to Queen El...,Medieval,Roman
80,Napoleonic-Era Field Kitchen Discovered in Gue...,Modern,Bronze Age


In [23]:
# Accuracy for the main test set (1 label)

ACCU = sklearn.metrics.accuracy_score(Easy_test['Label'], Easy_test['predicted'], normalize=True, sample_weight=None)
print('Easy_test Accuracy =',round((ACCU*100),2))

Easy_test Accuracy = 86.35


There are news stories which cannot be assigned to a single category as the content shows an overlap of 2 categories or more. Each news piece in the test data set below has been given 2 labels only for simplicity even though some of them are relevant to 3 categories. The news stories below are considered much more difficult to classify than the single-label test data set.

In [24]:
# A different test data set with 2 classes assgined to each news story
Hard_test = pd.read_csv('ARCHAIC_2labels.csv')

# Assign the primary label first (not necessarily the dominant class)
Hard_test, Hard_over_384 = col_label(Hard_test)

In [25]:
# Set up the numerical label column for the 2nd category
Hard_test['2nd_Label'] = 0 # want integers only
pair = []

for i in Hard_test.index:
      
  Hard_test.at[i,'2nd_Label'] = CL[Hard_test.at[i,'2nd category']]
  b = Hard_test.at[i,'2nd_Label']
  a = Hard_test.at[i,'Label']
  twin = [a,b]
  pair.append(twin)

# Add a column with a list of two numerical labels for each news piece
Hard_test['twin label'] = pair

In [26]:
Hard_input_ids,Hard_attention_masks, Hard_labels = Encode(Hard_test)
Hard_test_pred = bert_model([Hard_input_ids,Hard_attention_masks])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [27]:
# Collect predicted twin labels using the argsort function. The last two positions in the argsort results give max. and 2nd highest probability values
hd_predicted =[]

for l in range(len(Hard_test)):
  twin_index= (np.argsort(Hard_test_pred[0][l]))[-2 :]
  hd_predicted.append(twin_index)

Hard_test['predicted'] = hd_predicted  

In [28]:
# Rearrange the order of columns in the table for readability
Hard_test = Hard_test[['url','clean text','category','2nd category','Label','2nd_Label','twin label','predicted']]
Hard_test[['clean text','category','2nd category','Label','2nd_Label','twin label','predicted']]

,clean text,category,2nd category,Label,2nd_Label,twin label,predicted
0,"Finds Spanning 4,000 Years Unearthed in Southe...",Neolithic,Iron Age,5,10,"[5, 10]","[5, 10]"
1,Genome Study Yields New Date for Arrival of TB...,Medieval,Neolithic,13,5,"[13, 5]","[13, 5]"
2,Early Christian Chalice Unearthed in Northern ...,Roman,Medieval,2,13,"[2, 13]","[13, 2]"
3,Multidisciplinary Study Tracks Spread of Rice ...,East Asia,South&Southeast Asia,9,1,"[9, 1]","[9, 1]"
4,Possible Ancient Longboat Recovered in Ireland...,Neolithic,Medieval,5,13,"[5, 13]","[13, 5]"
5,Three Well-Preserved Ancient Boats Unearthed i...,Roman,Medieval,2,13,"[2, 13]","[10, 2]"
6,Leather Balls Found in Ancient Graves in North...,Bronze Age,East Asia,4,9,"[4, 9]","[4, 9]"
7,Volunteers Spot New Sites in Aerial Images of ...,Roman,Medieval,2,13,"[2, 13]","[10, 2]"
8,"Snake Altar Unearthed in Turkey. ANTALYA, TURK...",Roman,Iron Age,2,10,"[2, 10]","[4, 10]"
9,Study Examines Food and Gender in Bronze Age C...,Bronze Age,East Asia,4,9,"[4, 9]","[9, 4]"


In [29]:
# Dictionary reference
CL

{'Bronze Age': 4,
 'East Asia': 9,
 'Egyptian': 3,
 'Iron Age': 10,
 'Medieval': 13,
 'Modern': 8,
 'Native American': 0,
 'Near East': 12,
 'Neolithic': 5,
 'Paleolithic': 7,
 'Pre-Columbian': 11,
 'Pre-Human': 6,
 'Roman': 2,
 'South&Southeast Asia': 1}

In [30]:
# Custom scoring - both labels correct - 1, only one label correct - 0.5

score = 0

for j in Hard_test.index:
  if (Hard_test.at[j,'predicted'])[0] in Hard_test.at[j,'twin label']:
    score += 0.5
  else:
    pass
  if (Hard_test.at[j,'predicted'])[1] in Hard_test.at[j,'twin label']:
    score += 0.5
  else:
    pass

print('two-label accuracy =', round(score/(len(Hard_test))*100,2))


two-label accuracy = 67.76
